In [1]:
import requests
import json
import asyncio
from aiohttp import ClientSession
from web3 import Web3
from datetime import datetime
from pprint import pprint
import telebot

In [ ]:
# Addresses
# Wemix 3.0 network
wemix_addresses = [
    "0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F",
    "0xEaB8e0E07e80c35Fef600C988EC0121b1317696b",
    "0x446cd4BB55dD7cddD97D7EB8dBEF193Af3687273",
    "0xC5FFDA0067c63bC94D6Fa1248C2fc32eB3858a76"
]

# Ethereum network
eth_addresses = [
    "0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F",
    "0x62271357211D8A325165d61b60F0EdcACC05Bd3e",
    "0x6225C189f583fC2E17f5Be36A6A9db4d3978872D"
]

eth_contract_addresses = [
    "0x2c69095d81305F1e3c6ed372336D407231624CEa",
    "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"
]
    
# Polygon network
0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F

In [ ]:
# API parameters for WEMIX 3.0 network call
url_wemix = "https://api.wemix.com"

In [ ]:
# Calling wemix addresses
addresses = wemix_addresses

In [4]:
# Main function to call on the token balances from main rpcs. 
def get_balance(address, url, block_number=None):
    
    block_param = "latest" if block_number is None else hex(block_number)
    
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "eth_getBalance",
        "params": [address, block_param]
    }
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(url, json=payload, headers=headers)
    result = json.loads(response.text)['result']

    result_1 = int(result, 16)
    results_2 = result_1/1e18
    
    return results_2

In [ ]:
wemix_balance = []
for address in wemix_addresses:
    balance = get_balance(address, url_wemix, block_number_wemix)
    wemix_balance.append(balance)

In [ ]:
wemix_balance

In [ ]:
# Get block number asyncio version
async def get_block_number(session, url):
    payload = {
        "jsonrpc" : "2.0",
        "id" : 1,
        "method" : "eth_blockNumber",
        "params" : []
    }
    headers = {"Content-Type": "application/json"}
    async with session.post(url, json=payload, headers=headers) as response:
        result = await response.json()
        return int(result['result'], 16)

In [5]:
# Get block number
def get_block_number(url):
    payload = {
        "jsonrpc" : "2.0",
        "id" : 1,
        "method" : "eth_blockNumber",
        "params" : []
    }
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=payload, headers=headers)
    result = json.loads(response.text)["result"]
    return int(result, 16)

In [8]:
# Get block number timestamp asyncio version
async def get_block_timestamp(session, block_number, url):
    payload = {
        "jsonrpc" : "2.0",
        "id" : 1,
        "method" : "eth_getBlockByNumber",
        "params" : [hex(block_number), False]
    }
    headers = {"Content-Type": "application/json"}
    async with session.post(url, json=payload, headers=headers) as response:
        result = await response.json()
        return int(result["result"]["timestamp"], 16)

In [6]:
# Get block number timestamp
def get_block_timestamp(block_number, url):
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "eth_getBlockByNumber",
        "params": [hex(block_number), False]
    }
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=payload, headers=headers)
    result = json.loads(response.text)["result"]
    return int(result["timestamp"], 16)

In [ ]:
# Ethereum rpc node call for specific token balance asyncio version
async def get_token_balance(session, wallet, token_contract, url):
    function_signature = "0x70a08231"
    padded_wallet = "0" * (64 - len(wallet[2:])) + wallet[2:]
    data = function_signature + padded_wallet
    
    payload = {
        "jsonrpc" : "2.0",
        "id" : 1,
        "method": "eth_call",
        "params": [
                {
                "to" : token_contract,
                "data": data
            },
            "latest"
        ]
    }
    
    headers = {"Content-Type": "application/json"}
    
    async with session.post(url, json=payload, headers=headers) as response:
        result = await response.json()
        result_1 = int(result['result'], 16)
        if token_contract == "0x2c69095d81305F1e3c6ed372336D407231624CEa":
            result_2 = result_1/1e18
        if token_contract == "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48":
            result_2 = result_1/1e6
        
        return result_2

In [7]:
# Ethereum rpc node call for specific token balance
def get_token_balance(wallet, token_contract, url, block_number=None):
    function_signature = "0x70a08231"
    padded_wallet = "0" * (64 - len(wallet[2:])) + wallet[2:]
    data = function_signature + padded_wallet
    
    block_param = "latest" if block_number is None else hex(block_number)
    
    payload = {
        "jsonrpc" : "2.0",
        "id" : 1,
        "method": "eth_call",
        "params": [
                {
                "to" : token_contract,
                "data": data
            },
            block_param
        ]
    }
    
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=payload, headers=headers)
    result = json.loads(response.text)['result']
    result_1 = int(result, 16)
    
    """
    if token_contract == "0x2c69095d81305F1e3c6ed372336D407231624CEa":
        result_2 = result_1/1e18
    """
    
    if token_contract == "0x07865c6E87B9F70255377e024ace6630C1Eaa37F":
        result_2 = result_1/1e6
        
    return result_2

In [ ]:
# Main function to call the token addresses
async def main():
    # Wemix addresses
    wemix_addresses = [
        "0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F",
        "0xEaB8e0E07e80c35Fef600C988EC0121b1317696b",
        "0x446cd4BB55dD7cddD97D7EB8dBEF193Af3687273",
        "0xC5FFDA0067c63bC94D6Fa1248C2fc32eB3858a76"
    ]

    # Ethereum network
    eth_addresses = [
        "0x57192CCA8b8e4bEb77F3466C6D0550e64Cc53B0F",
        "0x62271357211D8A325165d61b60F0EdcACC05Bd3e",
        "0x6225C189f583fC2E17f5Be36A6A9db4d3978872D"
    ]

    # Ethereum token contracts
    eth_contract_addresses = [
        "0x2c69095d81305F1e3c6ed372336D407231624CEa",
        "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"
    ]

    # Ethereum node url
    url_eth = "https://mainnet.infura.io/v3/0c926d471842463fb7b7196054fc5bde"
    
    async with ClientSession() as session:
        tasks = []
        for address in eth_addresses:
            for token_contract in eth_contract_addresses:
                task = asyncio.ensure_future(get_token_balance(session, address, token_contract, url_eth))
                tasks.append(task)
                
        balances = await asyncio.gather(*tasks)
        
        for i, address in enumerate(eth_addresses):
            for j, token_contract in enumerate(eth_contract_addresses):
                balance = balances[i * len(eth_contract_addresses) + j]
                print(f"token balance of {address} for contract {token_contract}: {balance}")


In [ ]:
url_eth = "https://mainnet.infura.io/v3/0c926d471842463fb7b7196054fc5bde"
result_eth = []
for address in eth_addresses:
    for token_contract in eth_contract_addresses:
        token_balance = get_token_balance(address, token_contract, url_eth)
        result_eth.append(token_balance)

In [ ]:
result_eth

In [ ]:
# Get the block numbers and timestamps
block_number_wemix = get_block_number(url_wemix)
block_number_eth = get_block_number(url_eth)
block_timestamp = get_block_timestamp(block_number_eth, url_eth)
block_datetime = datetime.fromtimestamp(block_timestamp)

print(f"Block number (WEMIX): {block_number_wemix}")
print(f"Block number (ETH): {block_number_eth}")
print(f"Block timestamp: {block_timestamp} ({block_datetime})")

In [ ]:
# Main calling program

# URL for WEMIX 3.0 & ETH network call
url_wemix = "https://api.wemix.com"
url_eth = "https://mainnet.infura.io/v3/0c926d471842463fb7b7196054fc5bde"

# Get WEMIX balance and block number for wemix chain first
count = 0
wemix_balance = {}
previous_balance = []
block_number_wemix = get_block_number(url_wemix)
message_lines = [f"WEMIX Network"]
message_lines.append(f"Block number [WEMIX] :: {block_number_wemix}")
for address in wemix_addresses:
    balance = get_balance(address, url_wemix, block_number_wemix)
    #wemix_balance.append(balance)
    wemix_balance[address] = balance
    message_lines.append(f"{address} :: {balance} WEMIX ")

# Get balance on ETH wallet address
block_number_eth = get_block_number(url_eth)
message_lines.append(f"Ethereum Network")
message_lines.append(f"Block number [Ethereum] :: {block_number_eth}")
block_timestamp = get_block_timestamp(block_number_eth, url_eth)
block_datetime = datetime.fromtimestamp(block_timestamp)
result_eth = {}
for address in eth_addresses:
    for token_contract in eth_contract_addresses:
        token_balance = get_token_balance(address, token_contract, url_eth, block_number_eth)
        #result_eth.append(token_balance)
        result_eth[address] = token_balance
        if token_contract == '0x2c69095d81305F1e3c6ed372336D407231624CEa' :
            message_lines.append(f"{address} :: {token_balance} WEMIX")
        if token_contract == '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48' and token_balance != 0:
            message_lines.append(f"{address} :: {token_balance} USDC")

        
print(f"Block number (WEMIX): {block_number_wemix}")
print(wemix_balance)
print(f"Block number (ETH): {block_number_eth}")
print(result_eth)
print(f"Block timestamp: {block_timestamp} ({block_datetime})")

In [ ]:
wemix_balance

In [ ]:
result_eth

In [ ]:
# Main calling program

# URL for WEMIX 3.0 & ETH network call
url_wemix = "https://api.wemix.com"
url_eth = "https://mainnet.infura.io/v3/0c926d471842463fb7b7196054fc5bde"

# Get WEMIX balance and block number for wemix chain first
count = 0
previous_balance_wemix = {}
block_number_wemix = get_block_number(url_wemix)
message_lines = [f"WEMIX Network"]
message_lines.append(f"Block number [WEMIX] :: {block_number_wemix}")
for address in wemix_addresses:
    balance = get_balance(address, url_wemix, block_number_wemix)
    #wemix_balance.append(balance)
    previous_balance_wemix[address] = balance
    message_lines.append(f"{address} :: {balance} WEMIX ")

# Get balance on ETH wallet address
block_number_eth = get_block_number(url_eth)
message_lines.append(f"Ethereum Network")
message_lines.append(f"Block number [Ethereum] :: {block_number_eth}")
block_timestamp = get_block_timestamp(block_number_eth, url_eth)
block_datetime = datetime.fromtimestamp(block_timestamp)
previous_balance_eth = {}
for address in eth_addresses:
    for token_contract in eth_contract_addresses:
        token_balance = get_token_balance(address, token_contract, url_eth, block_number_eth)
        #result_eth.append(token_balance)
        previous_balance_eth[address] = token_balance
        if token_contract == '0x2c69095d81305F1e3c6ed372336D407231624CEa' :
            message_lines.append(f"{address} :: {token_balance} WEMIX")
        if token_contract == '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48' and token_balance != 0:
            message_lines.append(f"{address} :: {token_balance} USDC")

        
print(f"Block number (WEMIX): {block_number_wemix}")
print(previous_balance_wemix)
print(f"Block number (ETH): {block_number_eth}")
print(previous_balance_eth)
print(f"Block timestamp: {block_timestamp} ({block_datetime})")

In [ ]:
previous_balance_wemix

In [ ]:
previous_balance_eth

In [ ]:
# Compare the changes of the previous to new

for address in wemix_addresses:
    print(f'Wemix balance: {wemix_balance[address]}')
    print(f'Previous balance: {previous_balance_wemix[address]}')
    if wemix_balance[address] != previous_balance_wemix[address]:
        print(f'New transaction found on Block number: {block_number_wemix}!')
        transactions = get_transactions(address, url_wemix, block_number_wemix)
        print(transactions)
        


In [8]:
# Get transaction on WEMIX network
def get_transactions_wemix(address, url, block_number):
    transactions = []
    for block_index in range(block_number - 12, block_number+1):
    
        payload = {
            "jsonrpc": "2.0",
            "method": "eth_getBlockByNumber",
            "params": [hex(block_index), True],
            "id": 1
        }
        response = requests.post(url, json=payload)
        block = response.json()['result']
        
        for tx in block["transactions"]:
            if tx["from"].lower() == address.lower() or tx["to"].lower() == address.lower():
                transactions.append(tx)
    
    return transactions
            

In [9]:
# Get transaction from Ethereum network
def get_transactions_eth(address, url, block_number):
    transactions = []
    for block_index in range(block_number - 12, block_number+1):
    
        payload = {
            "jsonrpc": "2.0",
            "method": "eth_getBlockByNumber",
            "params": [hex(block_index), True],
            "id": 1
        }
        response = requests.post(url, json=payload)
        block = response.json()['result']
        
        for tx in block["transactions"]:
            if tx["from"] == address or tx["to"] == address:
                transactions.append(tx)
    
    return transactions

In [ ]:
address = '0x57192cca8b8e4beb77f3466c6d0550e64cc53b0f'
transaction = get_transactions_eth(address,url_eth, 17014626)
transaction

In [ ]:
block_number = transaction[0]["from"]
block_number

In [ ]:
# Telegram bot send function
def send_telegram_message(bot_token, chat_id, message):
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": message
    }
    response = requests.post(url, json=payload)
    return response.json()

In [ ]:
message_lines

In [ ]:
# Telegram bot 
#5961430353:AAENiehDUvjJ49K9Kw-fUAiGBhbV3jtPqrg

#Initialize bot:
bot_token = "5961430353:AAENiehDUvjJ49K9Kw-fUAiGBhbV3jtPqrg"
chat_id = '1628044387'

message = "\n".join(message_lines)
send_telegram_message(bot_token, chat_id, message)

In [ ]:
# Api for my bot
https://api.telegram.org/bot5961430353:AAENiehDUvjJ49K9Kw-fUAiGBhbV3jtPqrg/getMe
        
        

In [89]:
bot_token = "6220693290:AAHd2WGYNS5b9F9Cac_w7yWF4uVioETMv4I"

def get_updates(bot_token):
    url = f"https://api.telegram.org/bot{bot_token}/getUpdates"
    response = requests.get(url)
    return response.json()

updates = get_updates(bot_token)
pprint(updates)

{'ok': True,
 'result': [{'channel_post': {'chat': {'id': -1001828457128,
                                       'title': 'personal bot txn alert',
                                       'type': 'channel'},
                              'date': 1682240999,
                              'message_id': 239,
                              'sender_chat': {'id': -1001828457128,
                                              'title': 'personal bot txn alert',
                                              'type': 'channel'},
                              'text': 'test again'},
             'update_id': 642770389},
            {'channel_post': {'chat': {'id': -1001828457128,
                                       'title': 'personal bot txn alert',
                                       'type': 'channel'},
                              'date': 1682242146,
                              'message_id': 288,
                              'sender_chat': {'id': -1001828457128,
                           

In [ ]:
https://api.telegram.org/bot6280608213:AAHkbvZyK16FwgtaOK38MO40gN2jY3F578Q/getUpdates

In [10]:
def get_balance(address, url, block_number=None):
    
    block_param = "latest" if block_number is None else hex(block_number)
    
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "eth_getBalance",
        "params": [address, block_param]
    }
    headers = {"Content-Type": "application/json"}
    
    response = requests.post(url, json=payload, headers=headers)
    result = json.loads(response.text)['result']

    result_1 = int(result, 16)
    results_2 = result_1/1e18
    
    return results_2

In [6]:
address = '0x914B35276CB43CF64168Dc7FaB4EfBc423618014'
url = 'https://goerli.infura.io/v3/0c926d471842463fb7b7196054fc5bde'
block_number = 8851000

get_balance(address, url, block_number)

0.6176519939572677

In [10]:
get_block_number('https://goerli.infura.io/v3/0c926d471842463fb7b7196054fc5bde')

8851062

In [11]:
get_block_timestamp(8851062, 'https://goerli.infura.io/v3/0c926d471842463fb7b7196054fc5bde')

1681833852

In [19]:

address = '0x914B35276CB43CF64168Dc7FaB4EfBc423618014'
token_contract = "0x07865c6E87B9F70255377e024ace6630C1Eaa37F"
url = 'https://goerli.infura.io/v3/0c926d471842463fb7b7196054fc5bde'
block_number = 8851280
get_token_balance(address, token_contract, url, block_number)

316120.78324

In [76]:
def get_transactions_wemix(address, url, block_number, api_key):
    #transactions = []
    """
    for block_index in range(block_number - 3, block_number + 3):
        
        payload = {
            "jsonrpc": "2.0",
            "method": "eth_getBlockByNumber",
            "params": [hex(block_index), True],
            "id": 1
        }
        response = requests.post(url, json=payload)
        block = response.json()['result']

        
        for tx in block["transactions"]:
            if tx["from"].lower() == address.lower() or tx["to"].lower() == address.lower():
                transactions.append(tx)

        for tx in block["transactions"]:
            if tx["from"] == address.lower() or tx["to"] == address.lower():
                transactions.append(tx)
    """
    
    block_start = block_number
    block_end = block_number + 4
        
    url = f"https://api-goerli.etherscan.io/api?module=account&action=txlist&address={address}&startblock={block_start}&endblock={block_end}&page=1&offset=10&sort=asc&apikey={api_key}"
    response = requests.get(url)
    data = response.json()

    if data["message"] == "OK":
        transactions = data["result"]
        return transactions
    else:
        print("Error:", data["message"])
        return []
    

In [36]:
def get_transactions_eth(address, contract_address, url, block_number):
    print(f"blocknumber: {block_number}")
    print("inside get transaction eth")
    transactions = []
    transfer_method_signature = "0xa9059cbb"
    #for block_index in range(block_number - 5, block_number + 1):
    for block_index in range(block_number - 3, block_number + 3):
        print("inside block index for loop")
        print(f"blocknumber: {block_index}")
        payload = {
            "jsonrpc": "2.0",
            "method": "eth_getBlockByNumber",
            "params": [hex(block_index), True],
            "id": 1
        }
        response = requests.post(url, json=payload)
        block = response.json()['result']

        print("get block result to compare against the following address:")
        print(f"Address::: {address}")
        print("")
        for tx in block["transactions"]:

            print("now looking for transactions with address in that one block")
            print(tx)
            print("")

            if tx["to"] == contract_address.lower() and tx["input"].startswith(transfer_method_signature):
                #Extract the 'to' address from the input data
                to_address = "0x" + tx["input"][34:74]
                if tx["from"] == address.lower() or to_address == address.lower():
                    print("found one")
                    transactions.append(tx)
                    transaction_data = {
                        'from': tx['from'],
                        'to': to_address,
                        'hash': tx['hash']
                    }
                    return transaction_data
                
    

In [ ]:
            if tx["to"] == contract_address.lower() and tx["input"].startswith(transfer_method_signature):
                # Extract the 'to' address from the input data
                to_address = "0x" + tx["input"][34:74]

                if tx["from"] == address.lower() or to_address == address.lower():
                    print("found one")
                    transactions.append(tx)

In [69]:
address = '0x914B35276CB43CF64168Dc7FaB4EfBc423618014'
url = 'https://goerli.infura.io/v3/0c926d471842463fb7b7196054fc5bde'
block_number = 8871907
transactions = get_transactions_wemix(address, url, block_number)

Error: NOTOK


In [54]:
transactions[0]['hash']

'0xc2482cc9fa2c47fa0267ee21f55a497e0dbc7cb7fe624dc42de5aa14e2148204'

In [52]:
transactions[0]

{'accessList': [],
 'blockHash': '0xc42c27831cc3c55d7783925ba227246714a81517ca4f69b5d2389ef2617f3407',
 'blockNumber': '0x875fe3',
 'chainId': '0x5',
 'from': '0x914b35276cb43cf64168dc7fab4efbc423618014',
 'gas': '0x1832b',
 'gasPrice': '0x3f5476a49',
 'hash': '0xc2482cc9fa2c47fa0267ee21f55a497e0dbc7cb7fe624dc42de5aa14e2148204',
 'input': '0xa9059cbb00000000000000000000000044305dbe5566ae452788dd22081699ee9de13a720000000000000000000000000000000000000000000000000000001cbe8d1000',
 'maxFeePerGas': '0x4a817c800',
 'maxPriorityFeePerGas': '0x3f5476a00',
 'nonce': '0x13',
 'r': '0xa5179a331be597047df5a316af34b049e9343edef22c6ee30e8b008ae87582ce',
 's': '0x2f243cdf4aec5f35a25d7cc3864a7c9b7c96ff59e294401cc72bebf7561e189c',
 'to': '0x07865c6e87b9f70255377e024ace6630c1eaa37f',
 'transactionIndex': '0x1',
 'type': '0x2',
 'v': '0x1',
 'value': '0x0'}

In [43]:
address = '0x914B35276CB43CF64168Dc7FaB4EfBc423618014'
contract_address = '0x07865c6E87B9F70255377e024ace6630C1Eaa37F'
url = 'https://goerli.infura.io/v3/0c926d471842463fb7b7196054fc5bde'
block_number = 8871876
transactions = get_transactions_eth(address, contract_address, url, block_number)

blocknumber: 8871876
inside get transaction eth
inside block index for loop
blocknumber: 8871873
get block result to compare against the following address:
Address::: 0x914B35276CB43CF64168Dc7FaB4EfBc423618014

now looking for transactions with address in that one block
{'accessList': [], 'blockHash': '0x830219bbc39a706cd8f4457a822b8dd8a8a11c68a3ab01ab25167793c4b564c7', 'blockNumber': '0x875fc1', 'chainId': '0x5', 'from': '0x5a1f7d39a31eec32ca841da8524146a30cc4b6c2', 'gas': '0x30e4f', 'gasPrice': '0xb2d05e4a', 'hash': '0x7bebe68cf5f82fb0ef548dad9fdf372617bf22c61051a3229f5945e7a6ae1652', 'input': '0x8c3152e900000000000000000000000000000000000000000000000000000000000000200001000000000000000000000000000000000000000000000000000000068e0e0000000000000000000000005a1f7d39a31eec32ca841da8524146a30cc4b6c20000000000000000000000005a1f7d39a31eec32ca841da8524146a30cc4b6c2000000000000000000000000000000000000000000000000006a94d74f43000000000000000000000000000000000000000000000000000000000000000186a000

get block result to compare against the following address:
Address::: 0x914B35276CB43CF64168Dc7FaB4EfBc423618014

now looking for transactions with address in that one block
{'blockHash': '0x3d7e60b775ebdde0ce39a5445585403d3de7c834c8dd03915660b0f0248c3aee', 'blockNumber': '0x875fc2', 'chainId': '0x5', 'from': '0xd914134fa1a708c80ce293259c30a18d477c8730', 'gas': '0x5208', 'gasPrice': '0x6fc23ac00', 'hash': '0x828f5c3cdd1d53f05104152eefc970828ac43ba7c4df93087238eb00ef43a708', 'input': '0x', 'nonce': '0x164e6', 'r': '0xa3eedc32f0537f8790fec2e6e2b3a5e78edcabe63214edabc90c8f8289ff9b13', 's': '0x456aa037d72fd1afc282be105a4a356b608491998d9b52769226576d5539e205', 'to': '0xd914134fa1a708c80ce293259c30a18d477c8730', 'transactionIndex': '0x0', 'type': '0x0', 'v': '0x2e', 'value': '0x0'}

now looking for transactions with address in that one block
{'blockHash': '0x3d7e60b775ebdde0ce39a5445585403d3de7c834c8dd03915660b0f0248c3aee', 'blockNumber': '0x875fc2', 'chainId': '0x5', 'from': '0x52f4b76d629f

get block result to compare against the following address:
Address::: 0x914B35276CB43CF64168Dc7FaB4EfBc423618014

now looking for transactions with address in that one block
{'blockHash': '0x64dbb5219435ab8e6ee5b932e80990ea77279d6c0da62bf1e396f8e52f7f7a71', 'blockNumber': '0x875fc3', 'chainId': '0x5', 'from': '0xea28d002042fd9898d0db016be9758eeafe35c1e', 'gas': '0x5208', 'gasPrice': '0x15d3ef79800', 'hash': '0xf6a14d002dffd55bc958d4921e386541f4416811d86c9fcaca521bd3a2eb2ce1', 'input': '0x', 'nonce': '0x55f', 'r': '0x215b66765f4a505220f2ac713aac28862dc182ae9dcffec75d90a552353dd104', 's': '0x4dd9625a6688101d084858db8b915726785d774a171e14fed3c4baeb06e08c19', 'to': '0x8e77c0353c21f273e6e0bad3b198815303f9867c', 'transactionIndex': '0x0', 'type': '0x0', 'v': '0x2d', 'value': '0xebec21ee1da40000'}

now looking for transactions with address in that one block
{'accessList': [], 'blockHash': '0x64dbb5219435ab8e6ee5b932e80990ea77279d6c0da62bf1e396f8e52f7f7a71', 'blockNumber': '0x875fc3', 'chainId

get block result to compare against the following address:
Address::: 0x914B35276CB43CF64168Dc7FaB4EfBc423618014

now looking for transactions with address in that one block
{'blockHash': '0x08287bd207cb8e0bfd0a3d2849d9c21a5e23cc2a2e56123901182b6ab995362a', 'blockNumber': '0x875fc4', 'chainId': '0x5', 'from': '0x5023c8a019e4449bec125599f69615485ba32fa6', 'gas': '0x1e8480', 'gasPrice': '0xba43b7400', 'hash': '0x9fc40cc81ca771c1128720dca4d48b06f22ac4b533ee08a5e3b20f2dadb79dfe', 'input': '0x89a142490000000000000000000000000000000000000000000000000000000000005a4e0000000000000000000000002a2b94a6363fc0dbc3e10cce09051237519029ed000000000000000000000000000000000000000000000000000000000000060e00000000000000000000cf063da9fecc2ab6ed641a9e142d6eb00c901583000500000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000002200000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000

In [28]:
address.lower()

'0x914b35276cb43cf64168dc7fab4efbc423618014'

In [38]:
transactions

{'from': '0x44305dbe5566ae452788dd22081699ee9de13a72',
 'to': '0x914b35276cb43cf64168dc7fab4efbc423618014',
 'hash': '0x6e9e0ff9900b8efe9786a0d357c92f02ea4708954c24e84b0841637950a250a1'}

In [44]:
transactions

{'from': '0x44305dbe5566ae452788dd22081699ee9de13a72',
 'to': '0x914b35276cb43cf64168dc7fab4efbc423618014',
 'hash': '0x0ab63dfde1b9c7cc44e7c3e55ad4f7f574b12a707bca5f1d275d6a7b99cc2044'}

In [45]:
transactions['hash']

'0x0ab63dfde1b9c7cc44e7c3e55ad4f7f574b12a707bca5f1d275d6a7b99cc2044'

In [55]:
def get_transactions_etherscan(address):
    url = f"https://api.etherscan.io/api?module=account&action=txlist&address={address}&startblock=0&endblock=99999999&sort=asc&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    
    if data["message"] == "OK":
        transactions = data["result"]
        return transactions
    else:
        print("Error:", data["message"])
        return []

In [82]:
def get_transactions_eth_contract(address, contract, block_number, api_key):
    block_start = block_number -2
    block_end = block_number + 3
    url = f"https://api-goerli.etherscan.io/api?module=account&action=tokentx&contractaddress={contract}&address={address}&page=1&offset=100&startblock={block_start}&endblock={block_end}&sort=asc&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    
    if data["message"] == "OK":
        transactions = data["result"]
        return transaction
    else:
        print("Error: ", data ["message"])
        return []
    

In [84]:
address = '0x914b35276cb43cf64168dc7fab4efbc423618014'
api_key = 'YDYFQ26NV6BNQQ31Z64BCHW7YGHBVWJZF3'
contract = '0x07865c6E87B9F70255377e024ace6630C1Eaa37F'
block_number = 8871906
transaction = get_transactions_eth_contract(address, contract, block_number, api_key)

In [85]:
transaction

[{'blockNumber': '8871907',
  'timeStamp': '1682159844',
  'hash': '0xc2482cc9fa2c47fa0267ee21f55a497e0dbc7cb7fe624dc42de5aa14e2148204',
  'nonce': '19',
  'blockHash': '0xc42c27831cc3c55d7783925ba227246714a81517ca4f69b5d2389ef2617f3407',
  'transactionIndex': '1',
  'from': '0x914b35276cb43cf64168dc7fab4efbc423618014',
  'to': '0x07865c6e87b9f70255377e024ace6630c1eaa37f',
  'value': '0',
  'gas': '99115',
  'gasPrice': '17000000073',
  'isError': '0',
  'txreceipt_status': '1',
  'input': '0xa9059cbb00000000000000000000000044305dbe5566ae452788dd22081699ee9de13a720000000000000000000000000000000000000000000000000000001cbe8d1000',
  'contractAddress': '',
  'cumulativeGasUsed': '922390',
  'gasUsed': '65536',
  'confirmations': '222',
  'methodId': '0xa9059cbb',
  'functionName': 'transfer(address to, uint256 tokens)'}]

In [78]:
address = '0x914b35276cb43cf64168dc7fab4efbc423618014'
api_key = 'YDYFQ26NV6BNQQ31Z64BCHW7YGHBVWJZF3'
contract_address = '0x07865c6E87B9F70255377e024ace6630C1Eaa37F'
url = 'https://goerli.infura.io/v3/0c926d471842463fb7b7196054fc5bde'
block_number = 8871906
transaction = get_transactions_wemix(address, url, block_number, api_key)

In [79]:
transaction

[{'blockNumber': '8871907',
  'timeStamp': '1682159844',
  'hash': '0xc2482cc9fa2c47fa0267ee21f55a497e0dbc7cb7fe624dc42de5aa14e2148204',
  'nonce': '19',
  'blockHash': '0xc42c27831cc3c55d7783925ba227246714a81517ca4f69b5d2389ef2617f3407',
  'transactionIndex': '1',
  'from': '0x914b35276cb43cf64168dc7fab4efbc423618014',
  'to': '0x07865c6e87b9f70255377e024ace6630c1eaa37f',
  'value': '0',
  'gas': '99115',
  'gasPrice': '17000000073',
  'isError': '0',
  'txreceipt_status': '1',
  'input': '0xa9059cbb00000000000000000000000044305dbe5566ae452788dd22081699ee9de13a720000000000000000000000000000000000000000000000000000001cbe8d1000',
  'contractAddress': '',
  'cumulativeGasUsed': '922390',
  'gasUsed': '65536',
  'confirmations': '222',
  'methodId': '0xa9059cbb',
  'functionName': 'transfer(address to, uint256 tokens)'}]

In [2]:
from web3 import Web3

GOERLI_PROVIDER_URL = "https://goerli.infura.io/v3/0c926d471842463fb7b7196054fc5bde"
BLOCK_NUMBER = 8871490  # Replace with the block number you want to fetch

# Connect to the Goerli testnet
w3 = Web3(Web3.HTTPProvider(GOERLI_PROVIDER_URL))

# Check if connected to Ethereum
if not w3.is_connected():
    print("Failed to connect to Ethereum")
    exit()

# Get block
block = w3.eth.get_block(BLOCK_NUMBER)

# Get transaction hashes
transaction_hashes = block['transactions']

# Fetch and decode transactions
transactions = [w3.eth.get_transaction(tx_hash) for tx_hash in transaction_hashes]

for transaction in transactions:
    print(transaction)

AttributeDict({'accessList': [], 'blockHash': HexBytes('0x0129d7688232c76033bfcc55e7750171457fd4f4d4148c11beeea58fe879f424'), 'blockNumber': 8871490, 'chainId': 5, 'from': '0xd7584AEF6a45Ff5e2b6b55288BA3a8a74fc5F429', 'gas': 21000, 'gasPrice': 10000000167, 'hash': HexBytes('0x9bdcb2c416422a64a1bcfb4bdac8b6fed1c693991a2821f4ae37164b1487eaef'), 'input': '0x', 'maxFeePerGas': 30000000000, 'maxPriorityFeePerGas': 10000000000, 'nonce': 0, 'r': HexBytes('0x0c06aada56da0a48c9d624a9148d8260928d027f328c1131341e3a79ab693336'), 's': HexBytes('0x7bf9418dce19b9bab3693d849099d65de86512a705dfc6f9f11c5b4923664d6e'), 'to': '0x1218C9E22E5519Dd04332CF60b61dec3d242B372', 'transactionIndex': 0, 'type': 2, 'v': 0, 'value': 870848550000000000})
AttributeDict({'accessList': [], 'blockHash': HexBytes('0x0129d7688232c76033bfcc55e7750171457fd4f4d4148c11beeea58fe879f424'), 'blockNumber': 8871490, 'chainId': 5, 'from': '0xf096074a2F7F2216f33a8C872B2d328B759de7a7', 'gas': 77543, 'gasPrice': 3000000167, 'hash': HexB